En este fichero se entrena una red neuronal convolucional que pueda predecir características de un circuito en base a su trazado

Cargamos las imágenes etiquetadas y dividimos en entrenamiento y test


In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import f1_score



df=pd.read_csv(r"C:\Users\gonza\OneDrive\Escritorio\Universidad\4to curso\MINERIA DE DATOS\TRABAJO FINAL\DATOS EXTRAIDOS\imágenes_etiquetadas.csv")
# Dividir en train y test (80%-20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=100473601, stratify=df[["velocidad", "seguridad", "adelantamientos"]])




In [2]:
# Tamaño de las imágenes (ajústalo según tu modelo)
image_size = (128, 128)

def cargar_imagenes(df, carpeta_imagenes):
    imagenes = []
    etiquetas = []
    for _, row in df.iterrows():
        # Cargar y redimensionar la imagen
        ruta_imagen = os.path.join(carpeta_imagenes, row["nombre_imagen"])
        img = load_img(ruta_imagen, target_size=image_size)
        img_array = img_to_array(img) / 255.0  # Normalizar entre 0 y 1
        imagenes.append(img_array)
        
        # Guardar las etiquetas como vector
        etiquetas.append([row["velocidad"], row["seguridad"], row["adelantamientos"]])
    return np.array(imagenes), np.array(etiquetas)

# Cargar las imágenes y etiquetas
carpeta_imagenes = r"C:\Users\gonza\OneDrive\Escritorio\Universidad\4to curso\MINERIA DE DATOS\TRABAJO FINAL\FOTOS\FOTOS RED"
X_train, y_train = cargar_imagenes(train_df, carpeta_imagenes)
X_val, y_val = cargar_imagenes(test_df, carpeta_imagenes)

Vamos a definir una rejilla de valores de hiperparámetros para probar todas las combinaciones y poder escoger la mejor red

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import f1_score
from tqdm import tqdm

# Definir el grid de hiperparámetros, incluyendo 'epochs'
param_grid = {
    "num_filters": [(32, 64, 128), (64, 128, 256)],
    "dropout_rate": [0.3, 0.5],
    "learning_rate": [0.001, 0.0001],
    "batch_size": [32, 64,128],
    "epochs": [ 20,30,40]  # Añadido 'epochs' como un hiperparámetro
}


# Función para crear el modelo
def build_model(num_filters, dropout_rate, learning_rate):
    model = Sequential([
        Conv2D(num_filters[0], (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(num_filters[1], (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(num_filters[2], (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dense(3, activation='sigmoid')  # Salida para 3 características binarias
    ])
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']  # F1 Score se calculará manualmente
    )
    return model

# Almacenar resultados
best_f1 = 0
best_model = None
best_params = None



En este bucle se prueban todas las redes y se escoge la mejor

In [8]:
# Bucle para probar combinaciones de hiperparámetros
for num_filters in param_grid["num_filters"]:
    for dropout_rate in param_grid["dropout_rate"]:
        for learning_rate in param_grid["learning_rate"]:
            for batch_size in param_grid["batch_size"]:
                for epochs in param_grid["epochs"]:  # Iterar sobre el número de épocas
                    print(f"Entrenando con filtros {num_filters}, dropout {dropout_rate}, lr {learning_rate}, batch_size {batch_size}, epochs {epochs}")
                    
                    # Construir el modelo
                    model = build_model(num_filters, dropout_rate, learning_rate)
                    
                    # Crear una barra de progreso para las épocas
                    with tqdm(total=epochs, desc=f'Epochs {epochs}', ncols=100) as pbar:
                        # Callback para actualizar la barra de progreso
                        class TQDMProgressBar(tf.keras.callbacks.Callback):
                            def on_epoch_end(self, epoch, logs=None):
                                pbar.update(1)
                    
                        # Entrenar el modelo con el callback
                        history = model.fit(
                            X_train, y_train,
                            validation_data=(X_val, y_val),
                            epochs=epochs,  # Número de épocas variable
                            batch_size=batch_size,
                            verbose=0,  # Desactivado para usar la barra de progreso
                            callbacks=[TQDMProgressBar()]
                        )
                    
                    # Evaluar en el conjunto de validación
                    y_val_pred = (model.predict(X_val) > 0.5).astype(int)
                    f1 = f1_score(y_val, y_val_pred, average='weighted')  # Weighted para clases desbalanceadas
                    
                    print(f"F1 Score en validación: {f1}")
                    
                    # Actualizar el mejor modelo si este es mejor
                    if f1 > best_f1:
                        best_f1 = f1
                        best_model = model
                        best_params = {
                            "num_filters": num_filters,
                            "dropout_rate": dropout_rate,
                            "learning_rate": learning_rate,
                            "batch_size": batch_size,
                            "epochs": epochs
                        }


# Guardar el mejor modelo
best_model.save("mejor_modelo_circuitos.h5")
print(f"Mejor modelo guardado con F1 Score: {best_f1}")
print(f"Mejores hiperparámetros: {best_params}")


Entrenando con filtros (32, 64, 128), dropout 0.3, lr 0.001, batch_size 32, epochs 20


Epochs 20: 100%|████████████████████████████████████████████████████| 20/20 [03:08<00:00,  9.44s/it]


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step
F1 Score en validación: 0.7435032699572858

c:\Users\gonza\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Entrenando con filtros (32, 64, 128), dropout 0.3, lr 0.001, batch_size 32, epochs 30


...

c:\Users\gonza\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


F1 Score en validación: 0.6994664735919245
Entrenando con filtros (64, 128, 256), dropout 0.5, lr 0.0001, batch_size 128, epochs 40


Epochs 40: 100%|████████████████████████████████████████████████████| 40/40 [17:07<00:00, 25.70s/it]


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step


F1 Score en validación: 0.6964191491009597
Mejor modelo guardado con F1 Score: 0.7500320157090412
Mejores hiperparámetros: {'num_filters': (32, 64, 128), 'dropout_rate': 0.5, 'learning_rate': 0.001, 'batch_size': 128, 'epochs': 40}


Una vez que tenemos la mejor red, definimos una función que cargue una imagen y prediga sus características

In [9]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predecir_circuito(ruta_imagen, modelo, image_size=(128, 128)):
    # Cargar y preparar la imagen
    img = load_img(ruta_imagen, target_size=image_size)
    img_array = img_to_array(img) / 255.0  # Normalizar
    img_array = np.expand_dims(img_array, axis=0)  # Añadir dimensión batch

    # Predicción
    predicciones = modelo.predict(img_array)
    return predicciones[0]

# Cargar el modelo 
modelo = tf.keras.models.load_model("mejor_modelo_circuitos.h5")

Predecimos 5 circuitos españoles para ver sus características en caso de entrar en la Fórmula 1

In [38]:

# Tamaño de las imágenes usado durante el entrenamiento
image_size = (128, 128)


# Función para preparar una imagen
def preparar_imagen(ruta_imagen):
    img = load_img(ruta_imagen, target_size=image_size)  # Cargar la imagen y redimensionarla
    img_array = img_to_array(img) / 255.0  # Normalizar los píxeles a valores entre 0 y 1
    return np.expand_dims(img_array, axis=0)  # Añadir una dimensión para el batch

# Ruta de la carpeta con las imágenes
carpeta_imagenes = r"C:\Users\gonza\OneDrive\Escritorio\Universidad\4to curso\MINERIA DE DATOS\TRABAJO FINAL\FOTOS\FOTOS PRUEBA\New folder"
# Obtener la lista de imágenes
imagenes = [f for f in os.listdir(carpeta_imagenes) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Crear una lista para almacenar los resultados
resultados = []

# Recorrer todas las imágenes de la carpeta
for imagen_nombre in imagenes:
    ruta_imagen = os.path.join(carpeta_imagenes, imagen_nombre)
    # Preparar la imagen
    imagen_procesada = preparar_imagen(ruta_imagen)
    # Realizar la predicción
    prediccion = modelo.predict(imagen_procesada)
    prediccion_binaria = (prediccion > 0.5).astype(int)  # Convertir a valores binarios si es necesario
    # Guardar el resultado
    resultados.append({
        "nombre_imagen": imagen_nombre,
        "prediccion_continua": prediccion.tolist(),
        "prediccion_binaria": prediccion_binaria.tolist()
    })

# Crear mapeos inversos para decodificar las etiquetas
mapa_velocidad_inv = {0: "Lento", 1: "Rápido"}
mapa_seguridad_inv = {0: "Inseguro", 1: "Seguro"}
mapa_adelantamientos_inv = {0: "Pocos Adelantamientos", 1: "Muchos Adelantamiendos"}

# Mostrar los resultados con mapeo
for resultado in resultados:
    print(f"Imagen: {resultado['nombre_imagen']}")
    
    # Predicciones continuas (sin cambios)
    print(f"Predicción continua: {resultado['prediccion_continua']}")
    
    # Predicciones binarias (convertir de 1/0 a etiquetas)
    pred_binaria = resultado['prediccion_binaria'][0]  # Extraer lista de predicciones binarias
    print(f"Clasificación :  {mapa_velocidad_inv[pred_binaria[0]]}, "
          f" {mapa_seguridad_inv[pred_binaria[1]]}, "
          f" {mapa_adelantamientos_inv[pred_binaria[2]]}")
    print()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Imagen: CHESTE.png
Predicción continua: [[1.0, 0.9999998807907104, 0.9999698996543884]]
Clasificación :  Rápido,  Seguro,  Muchos Adelantamiendos

Imagen: CIRCUITO ALMERIA.png
Predicción continua: [[0.9999918341636658, 0.992418110370636, 0.0011154836975038052]]
Clasificación :  Rápido,  Seguro,  Pocos Adelantamientos

Imagen: CIRCUITO DE NAVARRA.png
Predicción continua: [[1.0, 0.9999995231628418, 0.8977177739143372]]
Clasificación :  Rápido,  Seguro,  Muchos Adelantamiendos

Imagen: CIRCUITO GUADIX.jpg
Predicción continua: [[0.999999463558197, 1.584191522852052e-05, 0.01137122604995966]]
Clasificación :  Rápido,  Inseguro,  Pocos Adelantamientos

Imagen: CIRCUITO VALENCIA.png
Predicción continua: [[0.00011638475552899763, 0.9985663890838623, 4.4724185954692075e-07]]
Clasificación :  Lento,  Seguro, 

A continuación vemos las características más típicas de los circuitos españoles

In [33]:
carpeta_imagenes = r"C:\Users\gonza\OneDrive\Escritorio\Universidad\4to curso\MINERIA DE DATOS\TRABAJO FINAL\FOTOS\FOTOS PRUEBA"  # Cambia esto a la ruta de tu carpeta
#carpeta_imagenes = r"C:\Users\gonza\OneDrive\Escritorio\Universidad\4to curso\MINERIA DE DATOS\TRABAJO FINAL\FOTOS\FOTOS PRUEBA\New folder"
# Obtener la lista de imágenes
imagenes = [f for f in os.listdir(carpeta_imagenes) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Crear una lista para almacenar los resultados
resultados = []

# Recorrer todas las imágenes de la carpeta
for imagen_nombre in imagenes:
    ruta_imagen = os.path.join(carpeta_imagenes, imagen_nombre)
    # Preparar la imagen
    imagen_procesada = preparar_imagen(ruta_imagen)
    # Realizar la predicción
    prediccion = modelo.predict(imagen_procesada)
    prediccion_binaria = (prediccion > 0.5).astype(int)  # Convertir a valores binarios si es necesario
    # Guardar el resultado
    resultados.append({
        "nombre_imagen": imagen_nombre,
        "prediccion_continua": prediccion.tolist(),
        "prediccion_binaria": prediccion_binaria.tolist()
    })


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [35]:
from collections import Counter


# Lista de predicciones
resultados

# Extraer predicciones continuas y binarias
predicciones_continuas = [item['prediccion_continua'][0] for item in resultados]
predicciones_binarias = [item['prediccion_binaria'][0] for item in resultados]

# Convertir a arrays para facilitar operaciones
predicciones_continuas = np.array(predicciones_continuas)
predicciones_binarias = np.array(predicciones_binarias)

# Calcular la media de las predicciones continuas
media_continuas = np.mean(predicciones_continuas, axis=0)

# Calcular la media de las predicciones binarias
media_binarias = np.mean(predicciones_binarias, axis=0)

# Calcular la moda de las predicciones binarias
moda_binarias = []
for i in range(predicciones_binarias.shape[1]):
    columna = predicciones_binarias[:, i]
    moda = Counter(columna).most_common(1)[0][0]
    moda_binarias.append(moda)

# Resultados
print("Media de las predicciones continuas:", media_continuas)
print("Moda de las predicciones binarias:", moda_binarias)


Media de las predicciones continuas: [0.87536575 0.74794224 0.24814561]
Moda de las predicciones binarias: [np.int64(1), np.int64(1), np.int64(0)]
